In [9]:
import pandas as pd

df = pd.read_csv("portfolio_articles_1.csv", sep="|", dtype='str')

In [32]:
df

,url,title,content,date,full_text
0,https://www.portfolio.hu/gazdasag/20250103/bal...,Balesettel és dugóval zárul az év első pénteke...,Frontális baleset miatt lezárták az M3-as autó...,2025/01/03,Balesettel és dugóval zárul az év első pénteke...
1,https://www.portfolio.hu/gazdasag/20241224/meg...,Megérkezett a kormány döntése a vendégmunkásokról,"A nemzetgazdasági miniszter arról döntött, hog...",2024/12/24,Megérkezett a kormány döntése a vendégmunkások...
2,https://www.portfolio.hu/gazdasag/20250103/veg...,Vége a száguldozásnak: Barcelonában keményen l...,A barcelonai városi tanács szigorú intézkedése...,2025/01/03,Vége a száguldozásnak: Barcelonában keményen l...
3,https://www.portfolio.hu/gazdasag/20250103/urs...,Ursula von der Leyen súlyos tüdőgyulladást kapott,"Ursula von der Leyen, az Európai Bizottság eln...",2025/01/03,Ursula von der Leyen súlyos tüdőgyulladást kap...
4,https://www.portfolio.hu/gazdasag/20250103/szo...,Szokatlan időjárási jelenség alakult ki a Bala...,Rendkívüli hőmérsékleti kontraszt alakult ki a...,2025/01/03,Szokatlan időjárási jelenség alakult ki a Bala...
...,...,...,...,...,...
20956,https://www.portfolio.hu/gazdasag/20220729/ori...,"Óriási károkat okoz a szárazság, az Alföldön e...","A szárazság jelentős károkat okoz, a terméskie...",2022/07/29,"Óriási károkat okoz a szárazság, az Alföldön e..."
20957,https://www.portfolio.hu/gazdasag/20220729/maj...,Majdnem 50 éves csúcson az osztrák infláció,Júliusban 47 éve a legmagasabb infláció alakul...,2022/07/29,Majdnem 50 éves csúcson az osztrák infláció. J...
20958,https://www.portfolio.hu/gazdasag/20220729/jo-...,Jó hírek érkeztek: brutálisan erős az olasz ga...,Olaszországban és Németországban is közzétetté...,2022/07/29,Jó hírek érkeztek: brutálisan erős az olasz ga...
20959,https://www.portfolio.hu/gazdasag/20220729/meg...,Megszólalt az EKB alelnöke: a gyenge euró növe...,Az euró leértékelődése az egyik legfőbb oka az...,2022/07/29,Megszólalt az EKB alelnöke: a gyenge euró növe...


In [14]:
df = df[
    (df["title"] != "Title not found") & 
    ~df["content"].fillna("").str.contains("Content not found") & 
    ~df["content"].fillna("").str.strip().eq("") & 
    ~df["content"].fillna("").str.contains("Error fetching content") & 
    df["error"].isnull()
]

In [17]:
df = df.drop("error", axis=1)

In [16]:
df[~df["error"].isnull()]

,url,title,content,date,error


In [19]:
df['full_text'] = df['title'] + ". " + df['content']
df = df.reset_index(drop=True)

In [24]:
df.to_csv("portfolio_articles_1.csv", sep="|", encoding='utf-8', index=False)

In [33]:
import pandas as pd
import asyncio
from concurrent.futures import ThreadPoolExecutor
from google import genai
from google.genai import types

client = genai.Client(api_key="")

# Function to translate each text
def translate_text(text):
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction="You are a translator that translates Hungarian text to English."
            ),
            contents=f"Please translate the following article to English in simple text format: {text}"
        )
        return response.text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Asynchronous wrapper for the translate function
async def translate_batch_async(batch):
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        tasks = [loop.run_in_executor(executor, translate_text, text) for text in batch["full_text"]]
        results = await asyncio.gather(*tasks)
    batch["translated_content"] = results
    return batch

# Main processing loop
async def main(df):
    try:
        df_translated = pd.read_csv("portfolio_articles_1_translated.csv", sep="|", encoding='utf-8')
        processed_ids = set(df_translated.index)
    except FileNotFoundError:
        df_translated = pd.DataFrame(columns=["full_text", "translated_content"])
        processed_ids = set()

    batch_size = 75  # Number of rows to process per batch
    start_index = max(processed_ids) + 1 if processed_ids else 0
    
    for i in range(start_index, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        batch = await translate_batch_async(batch)
        
        # Append to the result DataFrame
        df_translated = pd.concat([df_translated, batch], ignore_index=True)
        
        # Save to CSV after each batch
        df_translated.to_csv("portfolio_articles_1_translated.csv", index=False, sep="|", encoding='utf-8')
        
        print(f"Batch {i // batch_size + 1} processed and saved. Total articles translated: {len(df_translated)}")


# Run the asynchronous main function
await main(df)

C:\Users\tothk\AppData\Local\Temp\ipykernel_7360\3698830544.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 1 processed and saved. Total articles translated: 75


C:\Users\tothk\AppData\Local\Temp\ipykernel_7360\3698830544.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 2 processed and saved. Total articles translated: 150


C:\Users\tothk\AppData\Local\Temp\ipykernel_7360\3698830544.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


...

Batch 278 processed and saved. Total articles translated: 20850


C:\Users\tothk\AppData\Local\Temp\ipykernel_7360\3698830544.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 279 processed and saved. Total articles translated: 20925


C:\Users\tothk\AppData\Local\Temp\ipykernel_7360\3698830544.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 280 processed and saved. Total articles translated: 20961


In [42]:
df_proba = pd.read_csv("portfolio_articles_1_translated.csv", sep="|", dtype='str')

In [35]:
df_proba

,full_text,translated_content,url,title,content,date
0,Balesettel és dugóval zárul az év első pénteke...,The first Friday of the year ends with an acci...,https://www.portfolio.hu/gazdasag/20250103/bal...,Balesettel és dugóval zárul az év első pénteke...,Frontális baleset miatt lezárták az M3-as autó...,2025/01/03
1,Megérkezett a kormány döntése a vendégmunkások...,The government's decision regarding guest work...,https://www.portfolio.hu/gazdasag/20241224/meg...,Megérkezett a kormány döntése a vendégmunkásokról,"A nemzetgazdasági miniszter arról döntött, hog...",2024/12/24
2,Vége a száguldozásnak: Barcelonában keményen l...,The speeding is over: Barcelona is cracking do...,https://www.portfolio.hu/gazdasag/20250103/veg...,Vége a száguldozásnak: Barcelonában keményen l...,A barcelonai városi tanács szigorú intézkedése...,2025/01/03
3,Ursula von der Leyen súlyos tüdőgyulladást kap...,Ursula von der Leyen has a serious case of pne...,https://www.portfolio.hu/gazdasag/20250103/urs...,Ursula von der Leyen súlyos tüdőgyulladást kapott,"Ursula von der Leyen, az Európai Bizottság eln...",2025/01/03
4,Szokatlan időjárási jelenség alakult ki a Bala...,Unusual weather phenomenon developed at Lake B...,https://www.portfolio.hu/gazdasag/20250103/szo...,Szokatlan időjárási jelenség alakult ki a Bala...,Rendkívüli hőmérsékleti kontraszt alakult ki a...,2025/01/03
...,...,...,...,...,...,...
20956,"Óriási károkat okoz a szárazság, az Alföldön e...","The drought is causing enormous damage, and th...",https://www.portfolio.hu/gazdasag/20220729/ori...,"Óriási károkat okoz a szárazság, az Alföldön e...","A szárazság jelentős károkat okoz, a terméskie...",2022/07/29
20957,Majdnem 50 éves csúcson az osztrák infláció. J...,Austrian inflation is near a 50-year high. Acc...,https://www.portfolio.hu/gazdasag/20220729/maj...,Majdnem 50 éves csúcson az osztrák infláció,Júliusban 47 éve a legmagasabb infláció alakul...,2022/07/29
20958,Jó hírek érkeztek: brutálisan erős az olasz ga...,Good news: The Italian economy is brutally str...,https://www.portfolio.hu/gazdasag/20220729/jo-...,Jó hírek érkeztek: brutálisan erős az olasz ga...,Olaszországban és Németországban is közzétetté...,2022/07/29
20959,Megszólalt az EKB alelnöke: a gyenge euró növe...,The Vice-President of the ECB Speaks Out: Weak...,https://www.portfolio.hu/gazdasag/20220729/meg...,Megszólalt az EKB alelnöke: a gyenge euró növe...,Az euró leértékelődése az egyik legfőbb oka az...,2022/07/29


In [43]:
df_proba[df_proba["translated_content"].isnull()].count()

full_text             0
translated_content    0
url                   0
title                 0
content               0
date                  0
dtype: int64

In [40]:
# Function to translate each text
def translate_text(text):
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction="You are a translator that translates Hungarian text to English."
            ),
            contents=f"Please translate the following article to English in simple text format: {text}"
        )
        return response.text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Asynchronous wrapper for the translate function
async def translate_batch_async(batch):
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        tasks = [loop.run_in_executor(executor, translate_text, text) for text in batch["full_text"]]
        results = await asyncio.gather(*tasks)
    batch["translated_content"] = results
    return batch

# Main function to process and retry NaN translations
async def main():
    try:
        df_translated = pd.read_csv("portfolio_articles_1_translated.csv", sep="|", encoding='utf-8')
    except FileNotFoundError:
        df_translated = pd.DataFrame(columns=["full_text", "translated_content"])
    
    # First pass: Process untranslated rows
    untranslated_rows = df_translated["translated_content"].isna().sum()
    print(f"Untranslated rows found: {untranslated_rows}")

    if untranslated_rows > 0:
        to_retry = df_translated[df_translated["translated_content"].isna()]
        df_translated.loc[to_retry.index, "translated_content"] = (await translate_batch_async(to_retry))["translated_content"].values

        # Save the corrected translations
        df_translated.to_csv("portfolio_articles_1_translated.csv", index=False, sep="|", encoding='utf-8')
        print(f"Retry complete! Remaining untranslated rows: {df_translated['translated_content'].isna().sum()}")
    
    print("Translation process complete!")

# Run the asynchronous main function
await main()


Untranslated rows found: 221


C:\Users\tothk\AppData\Local\Temp\ipykernel_7360\20932026.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Retry complete! Remaining untranslated rows: 188
Translation process complete!


In [41]:
df_proba = pd.read_csv("portfolio_articles_1_translated.csv", sep="|", dtype='str')

df_proba.dropna(subset=['translated_content'], inplace=True)
df_proba.reset_index(drop=True, inplace=True)

df_proba.to_csv("portfolio_articles_1_translated.csv", index=False, sep="|", encoding='utf-8')